In [1]:

!pip install pycrf
!pip install sklearn-crfsuite

import spacy
import sklearn_crfsuite
from sklearn_crfsuite import metrics

model = spacy.load("en_core_web_sm")


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pycrf: filename=pycrf-0.0.1-py3-none-any.whl size=1875 sha256=501d379581826b7197f8e6ffb81cbb478804160ca4ad15fa455f9fe28d6e069f
  Stored in directory: c:\users\rohit_pandey1\appdata\local\pip\cache\wheels\e3\d2\c9\ba15b05ba596e2eafeb83c2903e79d634207367555aae8c7d2
Successfully built pycrf
  Obtaining dependency information for sklearn-crfsuite from https://files.pythonhosted.org/packages/b2/11/a8370dd6fce65f8f4e74a0adffae72be9db5799d8ed8ddbf84415356a764/sklearn_crfsuite-0.5.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for python-crfsuite>=0.9.7 from https://files.pythonhosted.org/packages/f3/4c/2aabe6f3c06a6e62fb4d80d0ed224e3d813b7fc5bc7d9aba52e724639268/python_crfsuite-0.9.11-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/301.9 kB ? eta -:--:--
   --- ----------------------------------- 30.7/301.9 kB 660.6 kB

ModuleNotFoundError: No module named 'spacy'

In [2]:
# Data Preprocessing 
def process_file(filename):
  input_file = open(filename, 'r')
  file_content = input_file.readlines() 
  input_file.close()

  out_lines = [] #To store list of sequences (sentences or labels)

  line_content = ""

  for word in file_content:
    word = word.strip() 
    if word == "": # If empty line, add the current sequence to out_lines
      out_lines.append(line_content)
      line_content = ""; # re-initialize
    else:
      if line_content: #if non-empty, add new word after space
        line_content += " "+word
      else:
        line_content = word # first word, no space required

  return out_lines

In [3]:
train_sentences = process_file('train_sent')
train_labels = process_file('train_label')
test_sentences = process_file('test_sent')
test_labels = process_file('test_label')
# Print the 5 sentences from the processed dataset
for i in range(5):
  print("Sentence:", train_sentences[i])
  print("Labels:", train_labels[i], "\n\n")
     

FileNotFoundError: [Errno 2] No such file or directory: 'train_sent'

In [4]:
# number of sentences
print("No. of lines in train_sentences:", len(train_sentences))
print("No. of lines in test_sentences:", len(test_sentences))

# number of lines of labels

print("No. of lines in train_labels:", len(train_labels))
print("No. of lines in test_labels:", len(test_labels))
     
# Extract those tokens which have NOUN or PROPN

concepts = {}

for sentences in (train_sentences, test_sentences):
  for sentence in sentences:
    processed_sentence = model(sentence) # Process each sentence by spacy model
    for token in processed_sentence:
      if(token.pos_ == 'NOUN' or token.pos_ == 'PROPN'): #check if the token is a noun
        concepts[token.text] = concepts.get(token.text, 0) + 1; #increase its frequency if it is noun

NameError: name 'train_sentences' is not defined

In [1]:
# Defining CFR 
def getFeaturesForOneWord(sentence, pos, pos_tags):
  word = sentence[pos]

  #Define 12 features with PoS tag as one of the features
  features = [
    'word.lower=' + word.lower(), # serves as word id
    'word[-3:]=' + word[-3:],     # last three characters
    'word[-2:]=' + word[-2:],     # last two characters
    'word.isupper=%s' % word.isupper(),  # is the word in all uppercase
    'word.isdigit=%s' % word.isdigit(),  # is the word a number
    'word.startsWithCapital=%s' % word[0].isupper(), # is the word starting with a capital letter
    'word.pos=' + pos_tags[pos]
  ]

  #Use the previous word also while defining features
  if(pos > 0):
    prev_word = sentence[pos-1]
    features.extend([
    'prev_word.lower=' + prev_word.lower(), 
    'prev_word.isupper=%s' % prev_word.isupper(),
    'prev_word.isdigit=%s' % prev_word.isdigit(),
    'prev_word.startsWithCapital=%s' % prev_word[0].isupper(),
    'prev_word.pos=' + pos_tags[pos-1]
  ])
  # Mark the begining and the end words of a sentence correctly in the form of features.
  else:
    features.append('BEG') # feature to track begin of sentence 

  if(pos == len(sentence)-1):
    features.append('END') # feature to track end of sentence

  return features
     

In [2]:

# Define a function to get features for a sentence using the 'getFeaturesForOneWord' function.
def getFeaturesForOneSentence(sentence):
  
  processed_sentence = model(sentence) #spacy is applied to sentence
  
  pos_tags = [] #correctly identify pos tags
  for token in processed_sentence:
    pos_tags.append(token.pos_)

  sentence_list = sentence.split() # List of words in sentence
  
  #Correctly calling getFeaturesForOneWord defined above
  return [getFeaturesForOneWord(sentence_list, pos, pos_tags) for pos in range(len(sentence_list))]

In [3]:
# Define a function to get the labels for a sentence.
def getLabelsInListForOneSentence(labels):
  return labels.split()

In [4]:
X_train = [getFeaturesForOneSentence(sentence) for sentence in train_sentences]
X_test = [getFeaturesForOneSentence(sentence) for sentence in test_sentences]

NameError: name 'train_sentences' is not defined

In [5]:

Y_train = [getLabelsInListForOneSentence(labels) for labels in train_labels]
Y_test = [getLabelsInListForOneSentence(labels) for labels in test_labels]
# Build the CRF model.

# Calling CRF 
crf = sklearn_crfsuite.CRF(max_iterations=100)

# Check that only X_train and Y_train are passed
crf.fit(X_train, Y_train)
diseases_and_treatments =  {} # dictionary with disease as key an list of treatments as value

for i in range(len(Y_pred)): # For each predicted sequence
  labels = Y_pred[i]

  disease = "";
  treatment = "";
  
  for j in range(len(labels)): # for each individual label in the sequence
    if labels[j] == 'O': # ignore if label is O -- other
      continue

    if(labels[j] == 'D'): # Label D indicates disease, so add the corresponding word from test sentence to the disease name string
      disease += test_sentences[i].split()[j] + " "
      continue

    if(labels[j] == 'T'): # Label T indicates disease, so add the corresponding word from test sentence to the treatment name string
      #print(test_sentences[i].split()[j])
      treatment += test_sentences[i].split()[j] + " "

  disease = disease.strip() # to remove extraneous spaces
  treatment = treatment.strip()

  # add the identified disease and treatment to the dictionary
  # if it is a new disease, directly add the value
  # if the disease has been seen previously, get the treatment list
  # and add current treatment to the list.
  if disease is not "" and treatment is not "":
    if disease not in diseases_and_treatments.keys():
      diseases_and_treatments[disease] = [treatment]
    else:
      treatment_list = diseases_and_treatments.get(disease)
      treatment_list.append(treatment)
      diseases_and_treatments[disease] = treatment_list 
     
        

<>:37: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:37: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:37: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:37: SyntaxWarning: "is not" with a literal. Did you mean "!="?


NameError: name 'train_labels' is not defined

In [6]:
diseases_identified = list(diseases_and_treatments.keys())
index = 94 

print("Disease: ",diseases_identified[index])
print("Treatment:", diseases_and_treatments.get(diseases_identified[index]))

NameError: name 'diseases_and_treatments' is not defined